In [24]:
import pandas as pd

In [25]:
A = pd.read_csv('ATR.csv')
B = pd.read_csv('D_EW_B.csv')
C = pd.read_csv('D_EW_S.csv')
D = pd.read_csv('HHLL.csv')
E = pd.read_csv('LR_Explore.csv')
F = pd.read_csv('Pattern_Revv.csv')
G = pd.read_csv('SCTR_Trial.csv')
H = pd.read_csv('ZigZag.csv')

Data cleaning to rename each value of Buy into the name of the tinker instead of 1

In [26]:
# Ensure Date/Time is properly parsed as datetime
for df in [A, B, C, D, E, F, G, H]:
    df['Date/Time'] = pd.to_datetime(df['Date/Time'], errors='coerce')  # Convert to datetime and handle errors

<bound method NDFrame.head of           Ticker  Date/Time Buy_ATR
0      360ONE.NS  9/26/2024     ATR
1      1STCUS.BO  9/24/2024     ATR
2       ^CNXAUTO  9/26/2024     ATR
3    ^CNXPSUBANK  9/23/2024     ATR
4      ^CNXMETAL  9/24/2024     ATR
..           ...        ...     ...
400  ZBINTXPP.BO  9/27/2024     ATR
401  ZEEMEDIA.NS  9/23/2024     ATR
402  ZEEMEDIA.NS  9/26/2024     ATR
403  ZSARACOM.BO  9/27/2024     ATR
404     ZUARI.NS  9/27/2024     ATR

[405 rows x 3 columns]>

In [27]:
# Renaming columns and performing necessary transformations
A.rename(columns={'Buy': 'Buy_ATR'}, inplace=True)
A['Buy_ATR'] = A['Buy_ATR'].replace(1, 'ATR')

B.rename(columns={'Mega Buy by Larger Wave ': 'Buy_EW'}, inplace=True)
B['Buy_EW'] = 'EW'

E.rename(columns={'myBuy ': 'Buy_LR'}, inplace=True)
E['Buy_LR'] = E['Buy_LR'].replace(1, 'LR')

F.rename(columns={'BullBreak': 'Buy_BB'}, inplace=True)
F['Buy_BB'] = F['Buy_BB'].replace('bullish Breakout', 'BB')

H.rename(columns={'Buy':'Buy_ZZ'}, inplace=True)
H['Buy_ZZ'] = H['Buy_ZZ'].replace(1, 'ZZ')

# Map values in the 'Pattern' column to new values
pattern_mapping = {
    'Up Channel': '_UC',
    'Wedge': '_W',
    'Down Channel': '_DC',
    'Broadening Wedge': '_BW',
    'Ascending Triangle': '_AT',
    'Decending Triangle': '_DT'
    }
F['Pattern'] = F['Pattern'].replace(pattern_mapping)

array(['EW'], dtype=object)

In [28]:
# Merge tables
tables = [A, B, C, D, E, F, G, H]
final_table = pd.DataFrame()

for table in tables:
    # Debugging: check the number of rows in each table before melt
    print(f"Table before melt (rows): {len(table)}")

    if 'Ticker' in table.columns:
        temp = pd.melt(table, id_vars=['Date/Time', 'Ticker'], var_name='Field', value_name='Value')
    else:
        temp = pd.melt(table, id_vars=['Date/Time'], var_name='Field', value_name='Value')

# Debugging: check the number of rows after melt
print(f"Table after melt (rows): {len(temp)}")

final_table = pd.concat([final_table, temp], ignore_index=True)

# Pivot the table to the final structure
final_table = final_table.pivot_table(index=['Date/Time', 'Ticker'], columns='Field', values='Value', aggfunc='first').reset_index()

<bound method NDFrame.head of               Ticker  Date/Time  Close  Mega Sellby Larger Wave        SL
0          360ONE.NS  7/31/2024   1068                  1128.16    926.30
1          360ONE.NS  9/12/2024   1068                  1104.03   1054.50
2      21STCENMGM.NS  7/16/2024    109                    81.10     47.65
3      21STCENMGM.NS  9/11/2024    109                   140.01     74.35
4          1STCUS.BO  8/13/2024    122                   127.15     84.20
...              ...        ...    ...                      ...       ...
14440      ZIMLAB.BO   9/2/2024    113                   122.70    109.45
14441    ZSARACOM.BO  7/12/2024  23827                  9474.80   8799.20
14442    ZSARACOM.BO  7/30/2024  23827                  9848.50   8600.00
14443    ZSARACOM.BO  8/30/2024  23827                 17125.18   8700.00
14444    ZSARACOM.BO  9/23/2024  23827                 26441.24  14047.45

[14445 rows x 5 columns]>

In [29]:
# Debugging: check if any rows are dropped after the pivot
    print(f"Final table after pivot (rows): {len(final_table)}")

<bound method NDFrame.head of              Ticker  Date/Time    Close       hh       ll  FromLow  FromHigh
0         1STCUS.BO  9/20/2024   113.95   127.19    89.35      1.0       0.0
1         1STCUS.BO  9/27/2024   121.55   126.35    89.35      2.0       0.0
2     21STCENMGM.NS  9/20/2024   120.82   141.40    42.70      0.0       1.0
3     21STCENMGM.NS  9/27/2024   109.19   141.40   109.19      0.0       2.0
4      20MICRONS.NS  9/20/2024   301.65   348.00   285.40      0.0       4.0
...             ...        ...      ...      ...      ...      ...       ...
6869   ZYDUSLIFE.NS  9/27/2024  1075.95  1324.30  1038.55      0.0       0.0
6870   ZYDUSWELL.NS  9/20/2024  2080.65  2484.00  2055.50      0.0       0.0
6871   ZYDUSWELL.NS  9/27/2024  2013.00  2484.00  1961.25      0.0       0.0
6872   ZODJRDMKJ.BO  9/20/2024    89.60   134.99    86.95      0.0       3.0
6873   ZODJRDMKJ.BO  9/27/2024    90.00   134.99    85.75      0.0       0.0

[6874 rows x 7 columns]>

In [31]:
F.rename(columns={'BullBreak': 'Buy_BB'}, inplace=True)
F['Buy_BB'] = F['Buy_BB'].replace('bullish Breakout', 'BB')
F['Pattern'].unique()

array(['Wedge', 'Up Channel', 'Down Channel', 'Broadening Wedge',
       'Ascending Triangle', 'Decending Triangle', nan], dtype=object)

In [32]:
# Create the function to change the values 
def change_status(value):
    if value =='Wedge':
        return 'W'
    elif value == 'Up Channel':
        return 'UC'
    elif value == 'Down Channel':
        return 'DC'
    elif value == 'Broadening Wedge':
        return 'BW'
    elif value == 'Ascending Triangle':
        return 'AT'
    elif value == 'Decending Triangle':
        return 'DT'
    else:
        return value

In [33]:
#Apply the function on pattern from dataset F
F['Pattern'] = F['Pattern'].apply(change_status)
F['Pattern'].unique()

array(['W', 'UC', 'DC', 'BW', 'AT', 'DT', nan], dtype=object)

In [34]:
H.rename(columns={'Buy':'Buy_ZZ'}, inplace=True)
H['Buy_ZZ'] = H['Buy_ZZ'].replace(1, 'ZZ')
H['Buy_ZZ'].unique()

array(['ZZ'], dtype=object)

In [38]:
# List of dataframes to merge
tables = [A, B, C, D, E, F, G, H]

# Start with an empty DataFrame for the final result
final_table = pd.DataFrame()

# Perform full outer join on Date/Time and create new rows for each Ticker
for table in tables:
    if 'Ticker' in table.columns:
        # Include 'Ticker' and 'Date/Time' in the melt operation
        temp = pd.melt(table, id_vars=['Date/Time', 'Ticker'], var_name='Field', value_name='Value')
    else:
        # If 'Ticker' is not present, use only 'Date/Time'
        temp = pd.melt(table, id_vars=['Date/Time'], var_name='Field', value_name='Value')

    final_table = pd.concat([final_table, temp], ignore_index=True)

# Pivot the final table to get back the desired structure
final_table = final_table.pivot_table(index=['Date/Time', 'Ticker'], columns='Field', values='Value', aggfunc='first').reset_index()

# Re-arrange columns to have 'Ticker' as the first column
cols = ['Ticker', 'Date/Time'] + [col for col in final_table.columns if col not in ['Ticker', 'Date/Time']]
final_table = final_table[cols]

# Fill NaN values with empty strings for the relevant columns
final_table[['Buy_ATR', 'Buy_EW', 'Buy_BB', 'Buy_LR', 'Pattern']] = final_table[['Buy_ATR', 'Buy_EW', 'Buy_BB', 'Buy_LR', 'Pattern']].fillna('')

# Create the 'Buy' column by concatenating the values from relevant columns
final_table['Buy'] = final_table['Buy_ATR'].astype(str) + final_table['Buy_EW'].astype(str) + final_table['Buy_BB'].astype(str) + final_table['Buy_LR'].astype(str) + final_table['Pattern'].astype(str)

# Re-arrange columns to place 'Buy' in the desired order
cols = ['Ticker', 'Date/Time', 'Buy'] + [col for col in final_table.columns if col not in ['Ticker', 'Date/Time', 'Buy']]
final_table = final_table[cols]

# Save the result to a CSV file
final_table.to_csv('joined_tickers.csv', index=False)
